### 1. Local inference

In [ ]:
import gc, torch, sagemaker
from sagemaker.s3 import S3Downloader
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
session = sagemaker.session.Session()

training_job_name = "fsdp-codestral-2024-06-17-05-45-33-367"
estimator = sagemaker.estimator.Estimator.attach(training_job_name)
model_s3_path = estimator.model_data["S3DataSource"]["S3Uri"]
model_local_path = "/home/ubuntu/finetune-llms-on-aws/practise-fsdp/sft_cache/sm_model"

print(f"-"*25)
print(f"model_s3_path: {model_s3_path}")
print(f"model_local_path: {model_local_path}")

#### run only once

In [ ]:
# !mkdir -p $model_local_path

In [ ]:
# response = S3Downloader.download(
#     s3_uri=model_s3_path,                    # S3 URI where the trained model is located
#     local_path=model_local_path,             # local path where *.targ.gz is saved
#     sagemaker_session=session                # SageMaker session used for training the model
# )

In [ ]:
## if tokenizer wasn't saved
# tokenizer = AutoTokenizer.from_pretrained("mistral-community/Codestral-22B-v0.1")
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

# tokenizer.save_pretrained(model_local_path)

In [ ]:
# !aws s3 sync $model_local_path $model_s3_path

#### continue

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_local_path, trust_remote_code=True,
    # "mistral-community/Codestral-22B-v0.1"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_local_path,
    trust_remote_code=True,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
)

In [ ]:
prompt = "Recommend the top 3 code generation language models to use for Rust"
prompt = "How to solve high leverage AI research problems ? And give examples where AI research helped humanity make leaps of progress."

model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
generated_ids = model.generate(
    **model_inputs, max_new_tokens=1000, do_sample=True
)
print(tokenizer.batch_decode(generated_ids)[0])

### 2. Local inference with vllm

In [ ]:
# ! pip install vllm ray
from vllm import LLM, SamplingParams
import gc, torch

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# model_id = "mistral-community/Codestral-22B-v0.1" # original
model_id = "/home/ubuntu/finetune-llms-on-aws/practise-fsdp/sft_cache/sm_model" # finetuned

print(f"model_id: {model_id}")

In [ ]:
llm = LLM(model_id, tensor_parallel_size=4, dtype="bfloat16")

In [ ]:
# prompt = "Recommend the top 3 code generation language models to use for Rust"
prompt = "How to solve high leverage AI research problems ? And give examples where AI research helped humanity make leaps of progress."

sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=1024)

output = llm.generate(prompt, sampling_params)
print(output[0].outputs[0].text)

### 3. Host sagemaker inference endpoint

In [ ]:
import io, json, boto3, sagemaker, jinja2, pathlib
from sagemaker import Model, image_uris, serializers, deserializers

In [ ]:
role = sagemaker.get_execution_role()
session = sagemaker.session.Session()
region = session._region_name
jinja_env = jinja2.Environment()

# training_job_name = "fsdp-codestral-2024-06-17-05-45-33-367"
# estimator = sagemaker.estimator.Estimator.attach(training_job_name)
# model_s3_path = estimator.model_data["S3DataSource"]["S3Uri"]
model_s3_path = "s3://research-agi/mistral-community-codestral-22b-v0x1/runs/fsdp-codestral-2024-06-17-05-45-33-367/output/model/"
s3_code_prefix = "djl_inference"
tar_file = "djl_inference_code.tar.gz"

print(f'role: {role} region: {region}')
print(f"model_s3_path: {model_s3_path}")

#### model artifacts

In [ ]:
!mkdir -p djl_inference

In [ ]:
%%writefile djl_inference/serving.properties
engine=Python
option.model_id={{s3url}}
option.rolling_batch=vllm
option.dtype=bf16
option.tensor_parallel_degree=4
option.max_rolling_batch_size=1
option.model_loading_timeout=1800

In [ ]:
template = jinja_env.from_string(
    pathlib.Path("djl_inference/serving.properties").open().read()
)

pathlib.Path("djl_inference/serving.properties").open("w").write(
    template.render(s3url=model_s3_path)
)

!pygmentize djl_inference/serving.properties | cat -n

In [ ]:
%%sh
tar czvf djl_inference_code.tar.gz djl_inference/

In [ ]:
bucket = session.default_bucket()  # bucket to house artifacts
code_artifact = session.upload_data(tar_file, bucket, s3_code_prefix)

print(f"s3 code uploaded to: {code_artifact}")

In [ ]:
%%sh
rm -rf djl_inference
rm -rf djl_inference_code.tar.gz

#### deploy

In [ ]:
image_uri = image_uris.retrieve(
    framework="djl-deepspeed", 
    region=session.boto_session.region_name, 
    version="0.27.0"
)
instance_type = "ml.g5.12xlarge"
endpoint_name = sagemaker.utils.name_from_base("codestral-vllm")

print(f"endpoint_name: {endpoint_name}")

In [ ]:
%%time

model = Model(image_uri=image_uri, model_data=code_artifact, role=role)

model.deploy(
   initial_instance_count=1,
   instance_type=instance_type,
   endpoint_name=endpoint_name,
   container_startup_health_check_timeout=1800,
   # volume_size=300, # comment for g5
   endpoint_logging=True,
)

#### inference

In [ ]:
endpoint_name

In [ ]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=session,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

In [ ]:
%%time 

prompt = "How to solve high leverage AI research problems ? And give examples where AI research helped humanity make leaps of progress."

res = predictor.predict(
    {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens":128, 
            "do_sample":"true",
            # "model_name": "mistral-community/Codestral-22B-v0.1"
        }
    }
)
print(res["generated_text"])